## SILVER TO GOLD LAYER


In [ ]:
import pandas as pd
from global_paths import (
    TradeHistorySilverFilePath,
    TradeHistoryGoldFilePath,
    SymbolSilverFilePath,
)

### Gold Layer - Trade History


In [ ]:
# # read the csv file
# df_TradeHistory = pd.read_csv(TradeHistorySilverFilePath)

# # replace scrip code to compnay name
# df_Symbol = pd.read_csv(SymbolSilverFilePath)
# df_Symbol["scrip_code"] = df_Symbol["scrip_code"].astype(str)

# # Convert 'datetime' to datetime type
# df_TradeHistory["datetime"] = pd.to_datetime(df_TradeHistory["datetime"])

# # Merge df_TradeHistory with df_Symbol on the matching columns
# df_TradeHistory = df_TradeHistory.merge(
#     df_Symbol[["scrip_code", "symbol"]],
#     left_on="scrip_code",
#     right_on="scrip_code",
#     how="left",
# )

# # Assign the new column 'stock_name' in df_TradeHistory to the values from 'symbol'
# df_TradeHistory["stock_name"] = df_TradeHistory["symbol"].combine_first(df_TradeHistory["stock_name"])

# # sort the dataframe by date
# df_TradeHistory = df_TradeHistory.sort_values(by="datetime")


# data = list()

# for _, row in df_TradeHistory[:10].iterrows():
#     data.append(row.to_dict())

# print(data)
# exit()

# df_TradeHistory = pd.DataFrame(data)
# df_TradeHistory = df_TradeHistory.round(2)

# df_TradeHistory = df_TradeHistory[
#     [
#         "datetime",
#         "segment",
#         "stock_name",
#         "side",
#         "buy_quantity",
#         "buy_price",
#         "buy_amount",
#         "sell_quantity",
#         "sell_price",
#         "sell_amount",
#         "profit_loss_amount",
#         "holding_quantity",
#         "holding_price_avg",
#         "holding_amount",
#     ]
# ]

# # sort the dataframe by date
# df_TradeHistory = df_TradeHistory.sort_values(by=["segment", "stock_name", "datetime"])

# # Save the result as a csv file
# df_TradeHistory.to_csv(TradeHistoryGoldFilePath, index=None)
# df_TradeHistory.info()
# print("Gold Layer csv file for trade history successfully created at:")
# print(TradeHistoryGoldFilePath.resolve())

In [ ]:
class Portfolio:
    def __init__(self):
        self.stocks = dict()

    def trade(self, record: dict = None):
        stock_name = str(record.get("stock_name"))

        if stock_name not in self.stocks:
            self.stocks[stock_name] = Stock(stock_name)

        record.update(
            self.stocks[stock_name].trade(
                side=str(record.get("side")).upper(),
                amount=float(record.get("amount")),
                quantity=int(record.get("quantity")),
                price=float(record.get("price")),
            )
        )
        return record


class Stock:
    def __init__(self, stock_name):
        self.stock_name = stock_name
        self.holding_price = 0
        self.holding_quantity = 0
        self.holding_amount = 0

    def trade(
        self,
        side: str,
        amount: float,
        quantity: int,
        price: float,
    ):
        buy_price = 0
        buy_quantity = 0
        buy_amount = 0
        sell_price = 0
        sell_quantity = 0
        sell_amount = 0

        if side == "BUY":
            buy_price = price
            buy_quantity = quantity
            buy_amount = amount
            self.holding_quantity += buy_quantity
            self.holding_amount += buy_amount
        elif side == "SELL":
            sell_price = price
            sell_quantity = quantity
            sell_amount = amount
            buy_price = self.holding_price
            buy_amount = quantity * buy_price
            self.holding_quantity -= sell_quantity
            self.holding_amount -= buy_amount
        else:
            raise Exception(f"{side} was never excepected")

        # Update the holding price only if there is remaining quantity
        if self.holding_quantity != 0:
            self.holding_price = self.holding_amount / self.holding_quantity

        return {
            "buy_price": buy_price,
            "buy_quantity": buy_quantity,
            "buy_amount": buy_amount,
            "sell_price": sell_price,
            "sell_quantity": sell_quantity,
            "sell_amount": sell_amount,
            "holding_price": self.holding_price,
            "holding_quantity": self.holding_quantity,
            "holding_amount": self.holding_quantity,
        }


portfolio = Portfolio()

data = [
    portfolio.trade(row)
    for row in [
        {
            "datetime": "2020-05-05 11:31:20",
            "exchange": "NSE",
            "segment": "EQ",
            "side": "BUY",
            "amount": 1165.5,
            "quantity": 10.0,
            "price": 116.55,
            "stock_name": "BHAGERIA",
        },
        {
            "datetime": "2020-05-19 14:44:06",
            "exchange": "NSE",
            "segment": "EQ",
            "side": "BUY",
            "amount": 500.0,
            "quantity": 5.0,
            "price": 100.0,
            "stock_name": "BHAGERIA",
        },
        {
            "datetime": "2020-06-15 10:57:42",
            "exchange": "NSE",
            "segment": "EQ",
            "side": "SELL",
            "amount": 1695.0,
            "quantity": 15.0,
            "price": 113.0,
            "stock_name": "BHAGERIA",
        },
    ]
]